# Optimization

**Outline**

* [Introduction](#intro)
* [Linear Programming](#lp)
    * [Brewer's Problem](#example1)
        * [Shadow Price](#shadow)
        * [Implementation using python Pulp](#exmaple1_implement)
        * [Dual Problem](#dual)        
    * [Simplified Shelf Space Problem](#example2)
* [Some Tips and Definitions](#tips)
* [Piecewise Linear Programming](#piecewise)
* [Reference](#reference)

**Disclaimer**

This is a note taken from 2018 Fall MSiA 440 Optimization class - Prof. Michael S. Watson. Hence, many of the content below can be copied directly from the class material

---

## <a id="intro">Introduction</a>

This is the [definition from wikipedia](https://en.wikipedia.org/wiki/Constrained_optimization)


> In mathematical optimization, **constrained optimization** (in some contexts called constraint optimization) is the process of optimizing an objective function with respect to some variables in the presence of constraints on those variables. The objective function is either a cost function or energy function, which is to be minimized, or a reward function or utility function, which is to be maximized. Constraints can be either **hard constraints**, which set conditions for the variables that are required to be satisfied, or **soft constraints**, which have some variable values that are penalized in the objective function if, and based on the extent that, the conditions on the variables are not satisfied.

## <a id='lp'>Linear Programming</a>

> **What is it?**

* Quintessential tool for optimal allocation of scarce resources, among a number of competing activities Powerful and general problem-solving method
* Linear Programming is a optimization method, a method to achieve the best outcome (such as maximum profit or lowest cost) in a mathematical model whose requirements are represented by linear relationships.


> **Why significant?**

* One of the most important mathematical techniques of the 20th century (Nobel Prize in Economics in 1975)
* Fast commercial solvers
* Powerful modeling languages
* Embedded in many types of applications
* Forms the basis of Integer, Non-Linear Programming, and Heuristics
* Widely applicable to industry (When someone mentions “optimizing” a process, chances are they are referring to LP Probably used somewhere in the companies you are associated with)
* When the term “linear programming” was coined, the term “programming” referred to planning or scheduling within a large organization. It has nothing to do with what we now call computer programming.

> **How Do you Create a Linear Programming Optimization Model?**

* Take your real business problems and challenges...
* ...and break down your problem so you can model it...
    * Objective
    * Decisions
    * Constraints
* ...and use this to create a set of equations that become your model.

\begin{aligned}&{\text{Maximize}}&&\mathbf {c} ^{\mathrm {T} }\mathbf {x} \\&{\text{subject to}}&&A\mathbf {x} \leq \mathbf {b} \\&{\text{and}}&&\mathbf {x} \geq \mathbf {0} \end{aligned}

> **The Components of an Optimization Problem are Worth Repeating**

* What makes one choice better than another? What **target** should be used to compare one set of choices to another? What is the **objective**? 
    * Objectives can be costs, revenue, profit, service, others...    
* What **resources** need to managed? How are they **limited**? What are the **constraints**? 
* What **choices** need to be in order to manage these resources? What are the **decision variables**?

> **Notes About Linear Programs**

* Have to be linear
    * Objective function and constraints must be written as linear equations
    * You can’t multiple decision variables together
    * You can’t divide by decision variables
    * You can’t use if-statements.  (I’m going to have you get started in Excel to build intuition– you really don’t want to use any formula in Excel except adding and multiplying by constants.

* The “linear” assumption seems to be limited
    * Let’s make sure we can do this before we try to tackle this assumption!

* Note:  We can solve these problems!  Two choices:
    * Choice One:  You might have to make some assumptions or create work-arounds because of linear assumptions, but you get a reliable answer
    * Choice Two:  You don’t use limiting linear assumptions, but you don’t have a way to solve the problem
* I think you’ll find:  the linear assumptions aren’t so bad!


> <a id='example1'>**Brewers Problem**</a>

Let's use a simple example to get a feeling of it! Here we go:

You are the owner of a small brewery. Since it's just the start of the business, we want to focus on brew 2 types of beer: Ale & Pilsner. These 2 types of beer would yield differet profit, also they need different combination of raw materials to brew it. 

The goal here is to decide how many Ale & Pilsner we want to make in order to have the maximum profit, assuming the only constraints here are raw materials. Here is the information

|Type| Corn(pounds) |  Hops(ounces)  | Malt(pounds) | Profit($) |
|------|------|------|
| Ale | 5 | 4 | 35 | 13 |
| Pilsner | 15 | 4 | 20 | 23 |
| Limit | 480 | 160 | 1190 | |

Let's change this problem into something we can model. Assume the number of Ale produced is **A** and the number of Pilsner produced is **B**

* **Objective**:
$$\text{maximize 23A+13B}$$

* **Constraints**
$$\begin{aligned}&{\text{Corn: 5A  + 15B <= 480}} \\&{\text{Hops:  4A  + 4B  <= 160}} \\&{\text{Malt:  35A + 20B <= 1190}}\end{aligned}$$

* **Decisions**:
\begin{cases}
    A  = ? \\
    B  = ?
  \end{cases}

> **<a id='shadow'>Shadow Price</a>**

In many occasions, we may want to know "what if we have 10 more punds of Corn? How much it would effect our profit?" or simply "How much of each of the raw material that we should get more in order to earn more?"

**Shadow price** can tell us this type of information. It is defined as the __*marginal increase of the objective when the constraint is relaxed by 1 unit*__.

Note:
* Work for Linear Programs (not Integer)
* Provide information on the impact to the objective value of small changes to the constraint (or right hand side)
* show which constraint is tight constraint
* Can provide value on the relative value of relaxing tight constraints
* Does not necessarily
    * Provide information on multiple changes at once (should we use all the budget to get Hops or should we get more Hops and some Corn?)
    * Provide information on large changes (should we get 1000 more corns?)
    
Comments on Shadow price:
* If you know about Linear Programming, you should know about Shadow Prices and Dual Problems
    * Everyone covers it in the classroom
    * The shadow prices can be useful (see below)
    * Sometimes the dual problem has a more intuitive interpretation
    * Some sophisticated algorithms take advantage of dual problems- you should know they exist and what they mean

But...
* Shadow prices are only for small changes and for linear programs (and most problems have integer variables)
* This was a lot more useful when computing power was at a premium and linear programs ran slow (remember, since 1988 LP’s are more than 5 million times faster!!)
* People are finding that is easier to just run a variety of what-if scenarios


> **<a id='example1_implement'>Implement using Pulp from Python</a>**

In [12]:
# This model separates the data from the optimization model
from pulp import *

#Create a list of all the products
Products = ["Ale","Pilsner"]

#Create a dictionary of the prices for products
Prices = {"Ale": 13,
          "Pilsner": 23}

#Create a list of all the raw materials
RawMaterials = ["Corn","Hops","Malt"]

#Create a Dictionary of Amount of each Raw Material Available
RawAvailability = {"Corn": 480,
                  "Hops": 160,
                  "Malt": 1190}

#Create a list for amount used of each raw material for each product
AmountUsed = {"Ale": {"Corn": 5, "Hops": 4, "Malt": 35},
              "Pilsner":{"Corn": 15, "Hops": 4, "Malt": 20}
             }

In [15]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Ale and Pilsner",LpMaximize)

# Create --decision variables--
product_vars = LpVariable.dicts("Prods",Products,lowBound=0,upBound=None,cat=LpContinuous)

# The --objective function-- is added to 'prob' first
prob += lpSum([Prices[i]*product_vars[i] for i in Products]), "Total Revenue of Production Plan"

# We can enter the --constraints-- that relate to limited amount of material
for r in RawMaterials:
    prob += lpSum([product_vars[i]*AmountUsed[i][r] for i in Products]) <= RawAvailability[r], r
    
# The problem data is written to an .lp file
prob.writeLP("BeerAle.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Total Revenue of Plan = ", value(prob.objective))

for constraint in prob.constraints:
        # print(prob.constraints[constraint].name, prob.constraints[constraint].value() - prob.constraints[constraint].constant)
        print(prob.constraints[constraint].name,"Remaining Slack" , prob.constraints[constraint].value())        
        print(prob.constraints[constraint].name,"Original RHS Value" , prob.constraints[constraint].constant)
        print(prob.constraints[constraint].name,"Shadow Price" , prob.constraints[constraint].pi)        

Status: Optimal
Prods_Ale = 12.0
Prods_Pilsner = 28.0
Total Revenue of Plan =  800.0
Corn Remaining Slack 0.0
Corn Original RHS Value -480
Corn Shadow Price 1.0
Hops Remaining Slack 0.0
Hops Original RHS Value -160
Hops Shadow Price 2.0
Malt Remaining Slack -210.0
Malt Original RHS Value -1190
Malt Shadow Price -0.0


In [16]:
prob

Ale and Pilsner:
MAXIMIZE
13*Prods_Ale + 23*Prods_Pilsner + 0
SUBJECT TO
Corn: 5 Prods_Ale + 15 Prods_Pilsner <= 480

Hops: 4 Prods_Ale + 4 Prods_Pilsner <= 160

Malt: 35 Prods_Ale + 20 Prods_Pilsner <= 1190

VARIABLES
Prods_Ale Continuous
Prods_Pilsner Continuous

The result indicats that we should brew 13 barrels of Ale and 23 barrels of Pilzner, and this would yield the maximum revenue of 800. As for the raw materials, we would use all the corn and hops, but still have 210 pounds of Malt left.

In term of the raw material, if we still have budget, our first priority is to get more Hops.

In reality, there might be many other constraints come into play, such as
* **Time**: maybe pilsner can be done faster than Ale, and we need to hand in our products within certain amount of time. 
* **Laber**: there may be only a certain people know how to brew ale.
* **Demand**: Pilsner may be less popular than Ale, we may over produce it if we don't take this into account.
* **Budget**: the factory would must have the total amount of money that they can spend
* **Space**: Maybe the factory don't have so much space to put all the barrels.
* **Other**

> **<a id='dual'>Dual Problems</a>**

Every Linear Program has a Dual. In rough terms, the dual problem is the reverse of the Primal (Think of the original Linear Program as the Primal). The constraints become decision variables, the decision variables become constraints.

Here shows the duality of brewer's problem:

<img src="_pic/duality.png" style="width: 500px;height: 350px;"/>

> **<a id='example2'>Simplified Shelf Space Problem using Open Solver</a>**

In a supermarker, since people's decision would be effect by where the product is located, it naturally make sense that how the product is located would effect the final profit.

In this model we want to determine which product will go onto which slot on the shelf. For each product, at each location, you know the expected daily profit as seen in this table.

<img src="_pic/ShelfSpaceTable.png" style="width: 800px;height: 200px;"/>

We need to determine what product should be slotted in which location. Only one product can go in each location and each product can only be slotted once.

Determine the profit if each item was slotted in its best possible location- not considering the overall feasibility of the solution. What does this number tell you from a business point of view?

The complete solution can be found [here in the google sheet](https://docs.google.com/spreadsheets/d/1Ey6uk2G4d2GYWT3bP6sIrPvSIsb9f9S__n9KQ3Tuv3k/edit?usp=sharing).

## <a id='tips'>Some Tips and Definitiion</a> 

> **Tips**

* Optimization Approaches: Getting The True Optimal Answer
    * **Enumeration**: when doing optimization problems, make sure we give user choices by listing out all the possible solutions and pick the best one
         * Pro’s
             * Easy
             * Works for small problems
             * Can help you explicit trade-offs
         * Con’s
             * Fails completely when the solution set is large. 
             * And, may real world problems have so many possible solutions, that enumeration is impossible
    * **Mathematical Optimization**: formulate the problem in the language of mathematics and use an approach that leads to a provable optimal solution
        * Pro’s
            * Can solve large problems
            * Guaranteed optimal solution
            * Robust with new constraints, variables, and data
        * Con’s
            * You have to be able to formulate the problem and may need to make assumptions to do so
            * Can be slow            

> **Definition**

* **Integer Programming**: like linear programming, except some of the variables can be forced to be binary– 0 or 1, or just an integer value
    * Where it seems like you would need integer variables
        * You must make an integer amount of product (120 cans, not 120.2)
        * You must order an integer amount of coils (263 vs 262.5)
        * You must order an integer amount of food (depending on how it is packaged)
        * Or, maybe you just don’t like to see decimal amounts 
        * And, many more cases...
    * Surprisingly, this is not the real reason IP exists. And, this use of IP usually makes the problems much harder, with limited benefits
    * Problems with IP:
        * We can not solve IP’s as fast as LP’s.
            * Problem is worse than not being just as fast
            * At the risk of oversimplifying, there are two basic type of optimization problems P and NP-Complete
        * We cannot solve IP’s as large as LP’s.
        * We do not get the sensitivity results for IP’s we get for  LP’s.
* **Non-Linear Optimization**: For some non-linear equations, there are techniques for finding the best solutions (think of the derivative in your Calculus class)
    * For some specific problems, there are other techniques
        * Options pricing
        * Queuing theory


## <a id='piecewise'>Piecewise Linear Programming</a>

Linear programming is good, but it's still linear. Linear formulations can be good approximations. To capture some aspects of the problem that follow cost structure as a curve, we can formulate the problem as piece-wise linear programs. 

Here we want to illustrate using a simple example.

* Company buys slabs of raw steel, does minor processing and resells the slabs
* They can sell for $5.50 per slab
* They have a budget of \$3,000 to buy slabs
* They can buy slabs at the following prices

<img src="_pic/piecewise.png" style="width: 400px;height: 120px;"/>

The solution can be found here: 
* [Case 1](https://docs.google.com/spreadsheets/d/1Ey6uk2G4d2GYWT3bP6sIrPvSIsb9f9S__n9KQ3Tuv3k/edit#gid=2125242938)
* [Case 2](https://docs.google.com/spreadsheets/d/1Ey6uk2G4d2GYWT3bP6sIrPvSIsb9f9S__n9KQ3Tuv3k/edit#gid=610776840)



## <a id="reference">Reference</a>

* [OpenSolver Website](https://opensolver.org/)
* [Youtube: Linear Programming - Shadow Price, Slack/Surplus calculations](https://www.youtube.com/watch?v=uaxOfTIC_pI)